# GERENCIAL ALFSTS/DIREP/EQPEM

In [1]:
import calendar
import os
import pandas as pd
import sys
from datetime import datetime
from sqlalchemy.sql import extract
from collections import defaultdict, OrderedDict
from sqlalchemy import create_engine
caminho_commons = os.path.join('..', '..', '..', 'ajna_docs', 'commons')
caminho_virasana = os.path.join('..', '..', '..', 'virasana')
sys.path.append(caminho_commons)
sys.path.append('..')
sys.path.append(caminho_virasana)
from ajna_commons.flask.conf import DATABASE, MONGODB_URI
from bhadrasana.models import engine, db_session
from bhadrasana.models import Usuario
from bhadrasana.models.ovr import OVR, EventoOVR
from bhadrasana.models.rvfmanager import lista_rvfovr
from IPython.display import HTML

2024-10-11 16:46:30,835 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  C:\Users\25052288840\PycharmProjects\bhadrasana2\notebooks\gerenciais\..\..\..\ajna_docs\commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  C:\Users\25052288840\PycharmProjects\bhadrasana2\notebooks\gerenciais\..\..\..\ajna_docs\commons\ajna_commons\flask\access.log


# Ações de Vigilância por mês

In [2]:
def get_fichas_vigilancia():
    lista_ovrs = db_session.query(OVR).join(EventoOVR)\
                                      .filter(OVR.setor_id.in_((1, 2, 3)))\
                                      .filter(OVR.tipooperacao.in_((7, )))\
                                      .filter(EventoOVR.fase >= 3)\
                                      .order_by(OVR.id, EventoOVR.create_date).all()
    lista_fichas = set()
    for ovr in lista_ovrs: # Não inclui OVR duas vezes
        lista_fichas.add(ovr)
    return lista_fichas

In [3]:
def get_peso_apreensoes(session, rvfs):
    peso = 0.
    for rvf in rvfs:
        for apreensao in rvf.apreensoes:
            try:
                peso += float(apreensao.peso)
            except TypeError:
                pass
    return peso

def get_valor_tgs(ovr):
    valor = 0.
    for tg in ovr.tgs:
        try:
            valor += float(tg.valor)
        except TypeError:
            pass
    return valor

def preenche_linha_operacao(operacao):
    linha = OrderedDict()
    linha['Ficha'] = operacao.id
    linha['Equipe'] = operacao.setor.nome
    recinto = 'Não informado'
    if operacao.recinto:
        recinto = operacao.recinto.nome
    linha['Local da Operação'] = recinto
    linha['Observações'] = operacao.observacoes
    rvfs = lista_rvfovr(db_session, operacao.id)
    achados = []
    for rvf in rvfs:
        if rvf.descricao:
            achados.append(rvf.descricao)
    linha['Achados'] = ' -*- '.join(achados)
    linha['Supervisor'] = str(operacao.cpfauditorresponsavel)
    linha['Responsável'] = str(operacao.responsavel_cpf)
    valortgs = get_valor_tgs(operacao)
    pesoapreensoes = get_peso_apreensoes(db_session, rvfs)
    servidores_set = set()
    for evento in operacao.historico:
        servidores_set.add(evento.user_name)
        if evento.fase >=3:
            data_encerramento = evento.create_date
    linha['Data Encerramento'] = data_encerramento
    flags =  ' '.join([flag.nome for flag in operacao.flags])
    linha['K9'] = 'K9' in flags
    linha['Lancha'] = 'Lancha' in flags
    linha['Drone'] = 'Drone' in flags
    servidores = []
    for cpf in servidores_set:
        servidor = db_session.query(Usuario).filter(Usuario.cpf == cpf).one()
        servidores.append({'cpf': servidor.cpf, 'nome': servidor.nome})
    linha['Servidores'] = servidores
    linha['Perdimento'] = valortgs
    linha['Apreensao'] = pesoapreensoes
    return linha


In [4]:
lista_fichas = get_fichas_vigilancia()
lista_final = []
for ficha in lista_fichas:
    linha_operacao = preenche_linha_operacao(ficha)
    lista_final.append(linha_operacao)
df = pd.DataFrame(lista_final)
df = df.sort_values('Data Encerramento')

In [5]:
df['AnoMes'] = df.apply(lambda x: str(x['Data Encerramento'].year) + str(x['Data Encerramento'].month).zfill(2), axis=1)

In [6]:
df_anomes = df.groupby(df.AnoMes).agg(
    quantidade=pd.NamedAgg(column='Ficha', aggfunc='count'),
    perdimentos=pd.NamedAgg(column='Perdimento', aggfunc='sum'),
    apreensoes=pd.NamedAgg(column='Apreensao', aggfunc='sum'),
    K9=pd.NamedAgg(column='K9', aggfunc='sum'),
    Lancha=pd.NamedAgg(column='Lancha', aggfunc='sum'),
    Drone=pd.NamedAgg(column='Drone', aggfunc='sum'),
).reset_index()
df_anomes

,AnoMes,quantidade,perdimentos,apreensoes,K9,Lancha,Drone
0,202011,1,0.0,0.0,0,0,0
1,202102,1,0.0,0.0,0,0,0
2,202103,1,0.0,0.0,0,0,0
3,202104,1,0.0,0.0,0,0,0
4,202112,1,0.0,0.0,0,0,0
5,202205,1,0.0,0.0,0,0,0
6,202206,2,0.0,0.0,0,0,0
7,202207,2,0.0,0.0,0,0,0
8,202209,1,0.0,0.0,0,0,0
9,202210,1,0.0,0.0,0,0,0


In [7]:
HTML(df.drop(columns=['AnoMes',]).to_html(index=False))

Ficha,Equipe,Local da Operação,Observações,Achados,Supervisor,Responsável,Data Encerramento,K9,Lancha,Drone,Servidores,Perdimento,Apreensao
1016,ALFSTS/DIREP,ECOPORTO SANTOS - OPERADOR TECONDI,"Operação regular de vigilância da EQPEM/DIREP/ALFSTS flagrou operação de abertura de Unidade de carga irregular, ao arrepio das normas e fora da área de cobertura de câmeras de vigilância.","Em operação regular de vigilância da Direp do Porto de Santos detectou-se operação irregular de abertura. Apesar de já estar armazenada desde 22/09/2020, a carga ainda não havia sido submetida a despacho aduaneiro. A unidade de carga estava aberta, conforme fotos anexas, em área distante das áreas de conferência. Estava presente a despachante Denise de Fátima Plácido, CPF 06218922898. -*- Em prosseguimento da operação de vigilância, foram encontradas outras unidades de carga, constantes de uma lista da Empresa BRACELL. Conforme fotos do local e fotos retiradas das câmeras de vigilância do COV ALFSTS, a cobertura de câmeras no local é precária, não atendendo as normas de Alfandegamento.Havia uma lista de dezenas de contêineres sendo abertos nesta área. A lista continha contêineres agrupados em ""motivos"", citando ""BRACELL"" e ""Inspeção de madeira"".",None,25052288840,2020-11-27 05:28:16,False,False,False,"[{'cpf': '02609824904', 'nome': 'André Feldman'}, {'cpf': '25052288840', 'nome': 'Ivan da Silva Brasílico'}]",0.0,0.0
1400,ALFSTS/DIREP/EQRIMP,EUDMARCO - PORTO SECO,Trata-se essa ficha de fiscalização realizada na EUDMARCO para verificação do seu sistema de cameras e escaner.,,None,13397296870,2021-02-25 16:17:20,False,False,False,"[{'cpf': '22983267890', 'nome': 'Edgar Palhari Lima'}, {'cpf': '13397296870', 'nome': 'Fabio Abdo Izzo'}]",0.0,0.0
1855,ALFSTS/DIREP/EQRIMP,EMBRAPORT - EMPRESA BRASILEIRA DE TERMINAIS PORTUA,CE COM 15 CONTEINERES QUE NÃO FORAM ESCANEADOS PELO TERMINAL E JÁ TEM DI NO VERDE DESEMBARAÇADA - 21/05211843 (DI) \r\nIMAGENS DISPONIBILIZADAS APÓS ESCANEAMENTO DOS 12 CTS FALTANTES EM 19/03/2021.,,None,13397296870,2021-03-19 10:26:57,False,False,False,"[{'cpf': '08937951860', 'nome': 'Andre Luiz Monteiro Pasqualini'}, {'cpf': '13397296870', 'nome': 'Fabio Abdo Izzo'}]",0.0,0.0
1937,ALFSTS/DIREP/EQRIMP,TERMARES IPA,Verificar se houve extravio de mercadoria devido ao tombamento do contêiner da carreta.,"O contêiner tombou no caminho entre a dpworld e a Termares. Devido a isso, o mesmo não foi escaneado. O lacre bate com o original e a abertura foi realizada sob a vigilância da câmera 546. Trata-se de uma carga de polímeros. Devido ao estado do contêiner a carga foi retirada e colocada sob guarda do Recinto Alfandegado. A abertura foi realizada com a presença do representante do importador Sr. Alexandre del Rosso Pires (despachante aduaneiro). A primeira vista foram encontrados dezoito sacos avariados.",None,13397296870,2021-04-19 11:16:54,False,False,False,"[{'cpf': '22983267890', 'nome': 'Edgar Palhari Lima'}, {'cpf': '13397296870', 'nome': 'Fabio Abdo Izzo'}]",0.0,0.0
3003,ALFSTS/DIREP/EQREXP,Brasil Terminal Portuario,"Em ronda da Equipe de Operações Marítimas, em 16 de novembro de 2021, às 14h30, verificou-se a operação de guindaste (Cábrea Pará) embarcado, de grande porte, junto ao costado lado mar do Navio MSC Athos, atracado no Brasil Terminal Portuário.\r\nTratava-se de operação, não autorizada pela Alfândega, de desembarque de duas cargas de grande porte do navio. O mestre da barcaça, Sr. Hélio Firmino de Oliveira, CPF nº 349977508-59, funcionário da Manobrasso declarou não possuir a autorização, expedida pela Alfândega da Receita Federal, para a realizar a operação.\r\nApós verificação de que todos os tripulantes da embarcação possuíam crachá ISPS Code, foi determinada a suspensão da operação.",Fotos relacionadas à ocorrência.,None,13397296870,2021-12-23 18:59:57,False,False,False,"[{'cpf': '29260847818', 'nome': 'Marcos Tadeu de Miranda'}, {'cpf': '13397296870', 'nome': 'Fabio Abdo Izzo'}]",0.0,0.0
1401,ALFSTS/DIREP/EQRIM